In [ ]:
from superconducting_qubit_tools.analysis.single_qubit_timedomain_analysis import InterleavedCoherenceAnalysis
from superconducting_qubit_tools.analysis.parallel_qubit_timedomain_analysis import ParallelInterleavedCoherenceAnalysis

for snapshot in datasets: # MODIFY THIS LOOP TO EXTRACT QUANTITIES OF INTEREST
    
    with open('20251024_parallel_coherence_monitoring_log.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Iteration', 'Timestamp', 'Qubit', 'T1', 'T2*', 'T2 Echo', 'Qubit Frequency'])

        # Main loop

        for  i in range(len(datasets)):

            # Timestamp
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            qubits = ['q0', 'q4']  # List of qubits to monitor. CHANGE

            for q in qubits:
                try:
                    t1 = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['T1'][q]['nominal_value'] 
                    t1_std_dev = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['T1'][q]['std_dev'] 

                    t2_star = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['T2*'][q]['nominal_value']
                    t2_star_std_dev = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['T2*'][q]['std_dev']

                    t2e = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['t2_echo'][q]['nominal_value']
                    t2e_std_dev = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['t2_echo'][q]['std_dev']


                    qubit_freq = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['qubit_frequency'][q]['nominal_value']
                    qubit_freq_std_dev = qf_data_h.load_quantities_of_interest(snapshot, analysis_name= 'ParallelInterleavedCoherenceAnalysis')['qubit_frequency'][q]['std_dev']
            
            # Print iteration summary
            print(f"[{timestamp}], Qubit = {q}, T1 = {t1:.2e}, T2* = {t2_star:.2e}, T2E = {t2e:.2e}, qubitfreq = {qubit_freq}")
    
            # Keep in CSV (This is the only place the data is stored permanently)
            writer.writerow([i + 1, timestamp, {q}, t1, t1_std_dev, t2_star, t2_star_std_dev, t2e, t2e_std_dev, qubit_freq])
            
            # Memory Optimization: Flush the write buffer to disk 💾
            file.flush()
            gc.collect() # Second, aggressive cleanup

    dataset = load_dataset(tuid=snapshot)
    analysis_obj = ParallelInterleavedCoherenceAnalysis(tuid=snapshot)
    analysis_obj.run(InterleavedCoherenceAnalysis)